# Model evaluation

In [1]:
from keras import backend as K
from keras.layers import Input
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Activation, Dense, Dropout, Lambda
from keras.layers.merge import Concatenate
from keras.models import Model, load_model
from keras.utils import np_utils
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from keras.optimizers import Adam

def load_vectors(vector_file):
    vec_dict = {}
    fvec = open(vector_file, "r")
    for line in fvec:
        image_name, image_vec = line.strip().split("\t")
        vec = np.array([float(v) for v in image_vec.split(",")])
        vec_dict[image_name] = vec
    fvec.close()
    return vec_dict

def batch_to_vectors(batch, vec_size, vec_dict):
    X1 = np.zeros((len(batch), vec_size))
    X2 = np.zeros((len(batch), vec_size))
    Y = np.zeros((len(batch), 2))
    for tid in range(len(batch)):
        X1[tid] = vec_dict[batch[tid][0]]
        X2[tid] = vec_dict[batch[tid][1]]
        Y[tid] = [1, 0] if batch[tid][2] == 0 else [0, 1]
    return ([X1, X2], Y)


def data_generator(triples, vec_size, vec_dict, batch_size=32):
    while True:
        # shuffle once per batch
        indices = np.random.permutation(np.arange(len(triples)))
        num_batches = len(triples) // batch_size
        for bid in range(num_batches):
            batch_indices = indices[bid * batch_size: (bid + 1) * batch_size]
            batch = [triples[i] for i in batch_indices]
            yield batch_to_vectors(batch, vec_size, vec_dict)

def evaluate_model(model, test_gen, triple_len):
    ytrue, ypred = [], []
    num_test_steps = triple_len // BATCH_SIZE
    for i in range(num_test_steps):
        (X1, X2), Y = next(test_gen)
        Y_ = model.predict([X1, X2])
        ytrue.extend(np.argmax(Y, axis=1).tolist())
        ypred.extend(np.argmax(Y_, axis=1).tolist())
    accuracy = accuracy_score(ytrue, ypred)
    print("\nAccuracy: {:.3f}".format(accuracy))
    print("\nConfusion Matrix")
    print(confusion_matrix(ytrue, ypred))
    print("\nClassification Report")
    print(classification_report(ytrue, ypred))
    return accuracy

Using TensorFlow backend.


In [2]:
IMG_DIR = '/home/nelssalminen/painters/data/'
OUTPUT_DIR = '/home/nelssalminen/painters/data/output/'
MODEL_DIR = '/home/nelssalminen/painters/data/scratch/models'

VECTOR_SIZE = 2048
VECTOR_FILE = os.path.join(OUTPUT_DIR, 'inceptionv3-vectors_alldata.tsv')
MODEL_FILE = os.path.join(MODEL_DIR, 'inceptionv3r2-cat-best.h5')

vec_dict = load_vectors(VECTOR_FILE)
model = load_model(MODEL_FILE)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
def get_triples(image_dir, dat, filename_label='filename', path_prefix=''):
    image_groups = {}
    for index, row in dat.iterrows():
        img_name = row[filename_label]
        group_name = row['artist']
        if group_name in image_groups:
            image_groups[group_name].append(path_prefix + img_name)
        else:
            image_groups[group_name] = [path_prefix + img_name]

    num_sims = 0
    image_triples = []
    group_list = sorted(list(image_groups.keys()))
    for i, g in enumerate(group_list):
        if num_sims % 100 == 0:
            print("Generated {:d} pos + {:d} neg = {:d} total image triples"
                  .format(num_sims, num_sims, 2 * num_sims))
        images_in_group = image_groups[g]
        sim_pairs_it = itertools.combinations(images_in_group, 2)
        # For each similar pair, generate a corresponding different pair
        for ref_image, sim_image in sim_pairs_it:
            image_triples.append((ref_image, sim_image, 1))
            num_sims += 1
            while True:
                j = np.random.randint(low=0, high=len(group_list), size=1)[0]
                if j != i:
                    break
            dif_image_candidates = image_groups[group_list[j]]
            k = np.random.randint(low=0, high=len(dif_image_candidates), size=1)[0]
            dif_image = dif_image_candidates[k]
            image_triples.append((ref_image, dif_image, 0))

    print("Generated {:d} pos + {:d} neg = {:d} total image triples"
          .format(num_sims, num_sims, 2 * num_sims))
    return image_triples

test_data_seen = pd.read_csv('/home/nelssalminen/painters/data/test_info-train_only.csv')
test_data_unseen = pd.read_csv('/home/nelssalminen/painters/data/test_info-test_only.csv')
test_data_mix = pd.read_csv('/home/nelssalminen/painters/data/test_info.csv')

test_triples_seen = get_triples(IMG_DIR, test_data_seen, 'new_filename');
test_triples_unseen = get_triples(IMG_DIR, test_data_unseen, 'new_filename');
test_triples_mix = get_triples(IMG_DIR, test_data_mix, 'new_filename');

Generated 0 pos + 0 neg = 0 total image triples
Generated 0 pos + 0 neg = 0 total image triples
Generated 1700 pos + 1700 neg = 3400 total image triples
Generated 23400 pos + 23400 neg = 46800 total image triples
Generated 137700 pos + 137700 neg = 275400 total image triples
Generated 151900 pos + 151900 neg = 303800 total image triples
Generated 181600 pos + 181600 neg = 363200 total image triples
Generated 255000 pos + 255000 neg = 510000 total image triples
Generated 261200 pos + 261200 neg = 522400 total image triples
Generated 279296 pos + 279296 neg = 558592 total image triples
Generated 0 pos + 0 neg = 0 total image triples
Generated 0 pos + 0 neg = 0 total image triples
Generated 0 pos + 0 neg = 0 total image triples
Generated 0 pos + 0 neg = 0 total image triples
Generated 65921 pos + 65921 neg = 131842 total image triples
Generated 0 pos + 0 neg = 0 total image triples
Generated 0 pos + 0 neg = 0 total image triples
Generated 118000 pos + 118000 neg = 236000 total image tripl

In [ ]:
VECTOR_SIZE = 2048
BATCH_SIZE = 256
#model.summary()

test_gen_seen = data_generator(test_triples_seen, VECTOR_SIZE, vec_dict, BATCH_SIZE)
final_accuracy = evaluate_model(model, test_gen_seen, len(test_triples_seen))

test_gen_unseen = data_generator(test_triples_unseen, VECTOR_SIZE, vec_dict, BATCH_SIZE)
final_accuracy = evaluate_model(model, test_gen_unseen, len(test_triples_unseen))

test_gen_mix = data_generator(test_triples_mix, VECTOR_SIZE, vec_dict, BATCH_SIZE)
final_accuracy = evaluate_model(model, test_gen_mix, len(test_triples_mix))

# for demo in demo_list:
# 	test_gen = data_generator(demo, VECTOR_SIZE, vec_dict, BATCH_SIZE)
# 	demo_accuracy = evaluate_model(model, test_gen)


Accuracy: 0.821

Confusion Matrix
[[253282  26014]
 [ 73812 205484]]

Classification Report
              precision    recall  f1-score   support

           0       0.77      0.91      0.84    279296
           1       0.89      0.74      0.80    279296

   micro avg       0.82      0.82      0.82    558592
   macro avg       0.83      0.82      0.82    558592
weighted avg       0.83      0.82      0.82    558592


Accuracy: 0.608

Confusion Matrix
[[42500 23420]
 [28295 37625]]

Classification Report
              precision    recall  f1-score   support

           0       0.60      0.64      0.62     65920
           1       0.62      0.57      0.59     65920

   micro avg       0.61      0.61      0.61    131840
   macro avg       0.61      0.61      0.61    131840
weighted avg       0.61      0.61      0.61    131840

